In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !pwd

Mounted at /content/drive
/content


In [17]:
!pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl.metadata (296 bytes)
Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [1]:
# # prompt: take me to the RA Data folder

# %cd /content/drive/MyDrive/RA Data

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
import os
from PIL import Image
import numpy as np
import torch.optim as optim


In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

In [3]:
# class DualTaskResNet(nn.Module):
#     def __init__(self, num_classes=1, pretrained=True):
#         super(DualTaskResNet, self).__init__()
#         # Load a pretrained ResNet
#         self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

#         # Remove the last fully connected layer
#         self.features = nn.Sequential(*list(self.resnet.children())[:-2])

#         # Upsampling layers for mask generation
#         self.upsample1 = self._make_upsample_block(2048, 1024)
#         self.upsample2 = self._make_upsample_block(1024, 512)
#         self.upsample3 = self._make_upsample_block(512, 256)
#         self.upsample4 = self._make_upsample_block(256, 64)
#         self.upsample5 = self._make_upsample_block(64, num_classes, final=True)

#         # Enhanced skeletonization layers
#         self.skeleton_decoder = nn.Sequential(
#             nn.Conv2d(num_classes, 32, kernel_size=3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(inplace=True),
#             ResidualBlock(32),
#             nn.Conv2d(32, 16, kernel_size=3, padding=1),
#             nn.BatchNorm2d(16),
#             nn.ReLU(inplace=True),
#             ResidualBlock(16),
#             nn.Conv2d(16, num_classes, kernel_size=1)
#         )

#     def _make_upsample_block(self, in_channels, out_channels, final=False):
#         layers = []
#         for i in range(3):  # Stack 3 ConvTranspose2d layers
#             if i == 0:
#                 layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2))
#             else:
#                 layers.append(nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
#             if not final:
#                 layers.append(nn.BatchNorm2d(out_channels))
#                 layers.append(nn.ReLU(inplace=True))
#         return nn.Sequential(*layers)

#     def forward(self, x, task='mask'):
#         x = self.features(x)
#         x = self.upsample1(x)
#         x = self.upsample2(x)
#         x = self.upsample3(x)
#         x = self.upsample4(x)
#         mask = self.upsample5(x)

#         if task == 'mask':
#             return torch.sigmoid(mask)
#         elif task == 'skeleton':
#             skeleton = self.skeleton_decoder(mask)
#             return torch.sigmoid(skeleton)

#     def freeze_encoder(self):
#         for param in self.features.parameters():
#             param.requires_grad = False

#     def unfreeze_encoder(self):
#         for param in self.features.parameters():
#             param.requires_grad = True

#     def freeze_decoder(self):
#         for layer in [self.upsample1, self.upsample2, self.upsample3, self.upsample4, self.upsample5]:
#             for param in layer.parameters():
#                 param.requires_grad = False

#     def unfreeze_decoder(self):
#         for layer in [self.upsample1, self.upsample2, self.upsample3, self.upsample4, self.upsample5]:
#             for param in layer.parameters():
#                 param.requires_grad = True

#     def finetune_for_skeleton(self):
#         # Freeze the encoder and decoder
#         self.freeze_encoder()
#         self.freeze_decoder()

#         # Unfreeze only the skeleton decoder
#         for param in self.skeleton_decoder.parameters():
#             param.requires_grad = True

#     def prepare_for_full_training(self):
#         # Unfreeze all layers for full training
#         self.unfreeze_encoder()
#         self.unfreeze_decoder()
#         for param in self.skeleton_decoder.parameters():
#             param.requires_grad = True

In [4]:
class DualTaskResNet(nn.Module):
    def __init__(self, num_classes=1, pretrained=True, task = "mask"):
        super(DualTaskResNet, self).__init__()
        # Load a pretrained ResNet
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2 if pretrained else None)

        # Remove the last fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])

        # Upsampling layers for output generation
        self.upsample1 = self._make_upsample_block(2048, 1024)
        self.upsample2 = self._make_upsample_block(1024, 512)
        self.upsample3 = self._make_upsample_block(512, 256)
        self.upsample4 = self._make_upsample_block(256, 64)
        self.upsample5 = self._make_upsample_block(64, num_classes, final=True)

        # Task-specific output layers
        self.mask_output = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.skeleton_output = nn.Conv2d(num_classes, num_classes, kernel_size=1)

    def _make_upsample_block(self, in_channels, out_channels, final=False):
        layers = []
        for i in range(3):  # Stack 3 ConvTranspose2d layers
            if i == 0:
                layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2))
            else:
                layers.append(nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
            if not final:
                layers.append(nn.BatchNorm2d(out_channels))
                layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x, task='mask'):
        x = self.features(x)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        x = self.upsample5(x)

        if task == 'mask':
            output = self.mask_output(x)
        elif task == 'skeleton':
            output = self.skeleton_output(x)
        else:
            raise ValueError("Task must be either 'mask' or 'skeleton'")

        return torch.sigmoid(output)

    def prepare_for_full_training(self):
        # Ensure all parameters are trainable
        for param in self.parameters():
            param.requires_grad = True

    def prepare_for_skeleton_finetuning(self):
        # Freeze all layers except the skeleton output layer
        for param in self.parameters():
            param.requires_grad = False
        for param in self.skeleton_output.parameters():
            param.requires_grad = True

In [5]:
# Test function
def test_dualtask_resnet():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create model instance
    model = DualTaskResNet(num_classes=1, pretrained=False).to(device)

    # Print model summary
    summary(model, (3, 224, 224))

    # Generate random input
    batch_size = 4
    input_tensor = torch.randn(batch_size, 3, 224, 224).to(device)

    # Test mask generation
    print("\nTesting mask generation:")
    mask_output = model(input_tensor, task='mask')
    print(f"Mask output shape: {mask_output.shape}")
    print(f"Mask output min: {mask_output.min().item():.4f}, max: {mask_output.max().item():.4f}")

    # Test skeletonization
    print("\nTesting skeletonization:")
    skeleton_output = model(input_tensor, task='skeleton')
    print(f"Skeleton output shape: {skeleton_output.shape}")
    print(f"Skeleton output min: {skeleton_output.min().item():.4f}, max: {skeleton_output.max().item():.4f}")

    # Test if outputs are different
    print("\nChecking if mask and skeleton outputs are different:")
    is_different = not torch.allclose(mask_output, skeleton_output)
    print(f"Outputs are different: {is_different}")



In [6]:
test_dualtask_resnet()

Using device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
            Conv2d-2         [-1, 64, 112, 112]           9,408
       BatchNorm2d-3         [-1, 64, 112, 112]             128
       BatchNorm2d-4         [-1, 64, 112, 112]             128
              ReLU-5         [-1, 64, 112, 112]               0
              ReLU-6         [-1, 64, 112, 112]               0
         MaxPool2d-7           [-1, 64, 56, 56]               0
         MaxPool2d-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]           4,096
           Conv2d-10           [-1, 64, 56, 56]           4,096
      BatchNorm2d-11           [-1, 64, 56, 56]             128
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
             ReLU-14

In [7]:
class DualTaskDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, skeleton_dir, image_transform=None, mask_transform=None):
        self.image_dir = os.path.normpath(image_dir)
        self.mask_dir = os.path.normpath(mask_dir)
        self.skeleton_dir = os.path.normpath(skeleton_dir)
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith('.jpeg') or f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.normpath(os.path.join(self.image_dir, img_name))

        # Construct the mask and skeleton paths
        base_name = os.path.splitext(img_name)[0]
        mask_name = base_name + '.png'
        skeleton_name = base_name + '.png'
        mask_path = os.path.normpath(os.path.join(self.mask_dir, mask_name))
        skeleton_path = os.path.normpath(os.path.join(self.skeleton_dir, skeleton_name))

        # Load the images
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        skeleton = Image.open(skeleton_path).convert('L')

        # Apply transforms
        if self.image_transform:
            image = self.image_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)
            skeleton = self.mask_transform(skeleton)

        return image, mask, skeleton


def get_data_loaders(image_dir, mask_dir, skeleton_dir, batch_size=32, train_split=0.7, val_split=0.15, test_split=0.15):
    # Define transforms for input images
    image_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5514, 0.4094, 0.3140], std=[0.1299, 0.1085, 0.0914])
    ])

    # Define transforms for masks and skeletons
    mask_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Create dataset
    full_dataset = DualTaskDataset(image_dir, mask_dir, skeleton_dir,
                                   image_transform=image_transform,
                                   mask_transform=mask_transform)

    # Calculate split sizes
    total_size = len(full_dataset)
    train_size = int(train_split * total_size)
    val_size = int(val_split * total_size)
    test_size = total_size - train_size - val_size

    # Split the dataset
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


In [8]:
def linear_loss_decay(epoch, total_epochs, start_weight=0.5, end_weight=1.0):
    return start_weight + (end_weight - start_weight) * (epoch / total_epochs)

def calculate_iou(pred, target, threshold=0.5):
    pred = (pred > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-6) / (union + 1e-6)  # Adding small epsilon to avoid division by zero

def calculate_pos_weight(train_loader):
    total_pixels = 0
    skeleton_pixels = 0
    for _, _, skeletons in train_loader:
        total_pixels += skeletons.numel()
        skeleton_pixels += skeletons.sum().item()
    neg_pos_ratio = (total_pixels - skeleton_pixels) / skeleton_pixels
    return torch.tensor([neg_pos_ratio])


In [13]:
def validate(model, val_loader, criterion, device, task):
    model.eval()
    val_loss = 0.0
    iou_sum = 0.0

    with torch.no_grad():
        for images, masks, skeletons in val_loader:
            images = images.to(device)
            targets = masks.to(device) if task == 'mask' else skeletons.to(device)

            outputs = model(images, task=task)
            loss = criterion(outputs, targets)

            val_loss += loss.item() * images.size(0)
            iou_sum += calculate_iou(outputs, targets)

    val_loss /= len(val_loader.dataset)
    val_iou = iou_sum / len(val_loader)

    return val_loss, val_iou

In [14]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

def train_model_with_finetuning(model, train_loader, val_loader, num_epochs, device):
    best_mask_iou = 0.0
    best_skeleton_iou = 0.0
    best_mask_model_weights = None
    best_skeleton_model_weights = None

    # Data collection for visualization
    mask_train_losses, mask_val_losses, mask_ious, mask_lrs = [], [], [], []
    skeleton_train_losses, skeleton_val_losses, skeleton_ious, skeleton_lrs = [], [], [], []

    # Phase 1: Train mask layers
    print("Phase 1: Training mask layers")
    model.prepare_for_full_training()

    criterion_mask = nn.BCELoss()
    optimizer_mask = torch.optim.Adam(model.parameters(), lr=0.001)

    mask_phase_epochs = num_epochs // 2
    constant_lr_epochs = mask_phase_epochs // 2
    cosine_annealing_epochs = mask_phase_epochs - constant_lr_epochs

    scheduler_mask = torch.optim.lr_scheduler.LambdaLR(optimizer_mask, lambda epoch: 1)  # Constant LR

    for epoch in tqdm(range(mask_phase_epochs), desc="Mask Training"):
        model.train()
        train_loss = 0.0

        for images, masks, _ in train_loader:
            images, masks = images.to(device), masks.to(device)

            optimizer_mask.zero_grad()

            outputs_mask = model(images)
            loss = criterion_mask(outputs_mask, masks)

            loss.backward()
            optimizer_mask.step()

            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        # Switch to cosine annealing after half the epochs
        if epoch == constant_lr_epochs - 1:
            scheduler_mask = CosineAnnealingLR(optimizer_mask, T_max=cosine_annealing_epochs)
        scheduler_mask.step()

        # Validation
        val_loss, mask_iou = validate(model, val_loader, criterion_mask, device, task='mask')

        print(f'Epoch {epoch+1}/{mask_phase_epochs}, Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, Mask IoU: {mask_iou:.4f}, '
              f'LR: {scheduler_mask.get_last_lr()[0]:.6f}')

        # Collect data for visualization
        mask_train_losses.append(train_loss)
        mask_val_losses.append(val_loss)
        mask_ious.append(mask_iou)
        mask_lrs.append(scheduler_mask.get_last_lr()[0])

        if mask_iou > best_mask_iou:
            best_mask_iou = mask_iou
            best_mask_model_weights = model.state_dict().copy()
            print(f'New best mask model saved with IoU: {best_mask_iou:.4f}')

    # Phase 2: Finetune skeleton model
    print("Phase 2: Finetuning skeleton model")

    # Load the best mask model before starting skeleton fine-tuning
    model.load_state_dict(best_mask_model_weights)
    model.prepare_for_skeleton_finetuning()

    # Calculate class weights for skeleton
    pos_weight = calculate_pos_weight(train_loader)
    criterion_skeleton = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))
    optimizer_skeleton = torch.optim.Adam(model.parameters(), lr=0.001)

    skeleton_phase_epochs = num_epochs - mask_phase_epochs
    constant_lr_epochs = skeleton_phase_epochs // 2
    cosine_annealing_epochs = skeleton_phase_epochs - constant_lr_epochs

    scheduler_skeleton = torch.optim.lr_scheduler.LambdaLR(optimizer_skeleton, lambda epoch: 1)  # Constant LR

    for epoch in tqdm(range(skeleton_phase_epochs), desc="Skeleton Finetuning"):
        model.train()
        train_loss = 0.0

        for images, _, skeletons in train_loader:
            images, skeletons = images.to(device), skeletons.to(device)

            optimizer_skeleton.zero_grad()

            outputs_skeleton = model(images)
            loss = criterion_skeleton(outputs_skeleton, skeletons)

            loss.backward()
            optimizer_skeleton.step()

            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        # Switch to cosine annealing after half the epochs
        if epoch == constant_lr_epochs - 1:
            scheduler_skeleton = CosineAnnealingLR(optimizer_skeleton, T_max=cosine_annealing_epochs)
        scheduler_skeleton.step()

        # Validation
        val_loss, skeleton_iou = validate(model, val_loader, criterion_skeleton, device, task='skeleton')

        print(f'Epoch {mask_phase_epochs + epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, Skeleton IoU: {skeleton_iou:.4f}, '
              f'LR: {scheduler_skeleton.get_last_lr()[0]:.6f}')

        # Collect data for visualization
        skeleton_train_losses.append(train_loss)
        skeleton_val_losses.append(val_loss)
        skeleton_ious.append(skeleton_iou)
        skeleton_lrs.append(scheduler_skeleton.get_last_lr()[0])

        if skeleton_iou > best_skeleton_iou:
            best_skeleton_iou = skeleton_iou
            best_skeleton_model_weights = model.state_dict().copy()
            print(f'New best skeleton model saved with IoU: {best_skeleton_iou:.4f}')

    # Collect all visualization data
    visualization_data = {
        'mask': {
            'train_losses': mask_train_losses,
            'val_losses': mask_val_losses,
            'ious': mask_ious,
            'lrs': mask_lrs
        },
        'skeleton': {
            'train_losses': skeleton_train_losses,
            'val_losses': skeleton_val_losses,
            'ious': skeleton_ious,
            'lrs': skeleton_lrs
        }
    }

    return model, best_mask_model_weights, best_skeleton_model_weights, visualization_data

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

image_dir = "DATA/IMG"
mask_dir = "DATA/MASK"
skeleton_dir = "DATA/SKELETON"
batch_size = 32

train_loader, val_loader, test_loader = get_data_loaders(image_dir, mask_dir, skeleton_dir, batch_size)

model = DualTaskResNet(num_classes=1, pretrained=True)
model.to(device)

model, best_mask_weights, best_skeleton_weights, vis_data = train_model_with_finetuning(
    model, train_loader, val_loader, num_epochs=100, device=device
)

# Save the best model
# Save the models
torch.save(best_mask_weights, 'best_mask_model.pth')
torch.save(best_skeleton_weights, 'best_skeleton_model.pth')

print("Training completed and best model saved.")



Using device: cuda
Phase 1: Training mask layers


Mask Training:   0%|          | 0/50 [01:43<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def visualize_training(visualization_data):
    fig, axs = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Training and Validation Metrics', fontsize=16)

    phases = ['mask', 'skeleton']
    colors = ['blue', 'red']

    for i, phase in enumerate(phases):
        # Plot training and validation loss
        axs[i, 0].plot(visualization_data[phase]['train_losses'], label='Train Loss', color=colors[0])
        axs[i, 0].plot(visualization_data[phase]['val_losses'], label='Val Loss', color=colors[1])
        axs[i, 0].set_title(f'{phase.capitalize()} Loss')
        axs[i, 0].set_xlabel('Epoch')
        axs[i, 0].set_ylabel('Loss')
        axs[i, 0].legend()

        # Plot IoU
        axs[i, 1].plot(visualization_data[phase]['ious'], label='IoU', color=colors[0])
        axs[i, 1].set_title(f'{phase.capitalize()} IoU')
        axs[i, 1].set_xlabel('Epoch')
        axs[i, 1].set_ylabel('IoU')
        axs[i, 1].legend()

        # Plot Learning Rate
        axs[i, 2].plot(visualization_data[phase]['lrs'], label='Learning Rate', color=colors[0])
        axs[i, 2].set_title(f'{phase.capitalize()} Learning Rate')
        axs[i, 2].set_xlabel('Epoch')
        axs[i, 2].set_ylabel('Learning Rate')
        axs[i, 2].legend()

    plt.tight_layout()
    plt.show()



In [ ]:
# Visulize Training
visualize_training(vis_data)

In [ ]:
test_loader_for_test = test_loader

# Evaluate on test set
test_loss, test_mask_iou, test_skeleton_iou, test_avg_iou = validate(trained_model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Mask IoU: {test_mask_iou:.4f}, '
        f'Test Skeleton IoU: {test_skeleton_iou:.4f}, Test Avg IoU: {test_avg_iou:.4f}')

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from torchvision.utils import save_image

def generate_skeletons(model, test_dataloader, save_dir, device='cuda'):
    model.eval()  # Set the model to evaluation mode
    os.makedirs(save_dir, exist_ok=True)  # Ensure the save directory exists

    with torch.no_grad():  # No need to compute gradients for inference
        for i, batch in tqdm(enumerate(test_dataloader), desc="Generating Skeletons"):
            # Unpack only the images from the batch (ignore other values)
            images = batch[0].to(device)

            # Generate skeleton predictions
            skeleton_preds = model(images, task='skeleton')

            # Apply a threshold to get binary skeletons
            skeleton_binary = (skeleton_preds > 0.5).float()

            # Save each predicted skeleton
            for j in range(skeleton_binary.size(0)):
                save_path = os.path.join(save_dir, f"skeleton_{i * test_dataloader.batch_size + j}.png")
                save_image(skeleton_binary[j], save_path)

                print(f"Skeleton saved at: {save_path}")





In [ ]:
# Load your trained model
model = DualTaskResNet(num_classes=1, pretrained=False)  # Adjust num_classes if necessary
model.load_state_dict(torch.load("best_skeleton_model.pth"))  # Load the trained weights
model = model.to('cuda')  # Send the model to the GPU (or use 'cpu')

# Load your test dataloader (assumes test_dataloader is defined)
# Example: test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Directory where skeletons will be saved
save_dir = "./skeleton_predictions"

# Call the skeleton generation function
generate_skeletons(model, test_loader, save_dir, device='cuda')

In [ ]:
for _, _, skeletons in train_loader:
    print(skeletons.min(), skeletons.max())